In [0]:
!pip install kafka-python #Instalación de Kafka

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
#Algunas librerías a emplear
from kafka import KafkaProducer
import requests
import time
import datetime
api_key = 'd65d7dda1cfde9f7a7f6f517d624d8a0c575f186647af620f9170e5bd5ef7092'
api_url = "https://min-api.cryptocompare.com/data/pricemulti?fsyms=ETH&tsyms=BTC,USD,EUR&api_key={api_key}"

## Ventanas deslizantes

Es un valor calculado de manera continua de la media aritmética durante un periodo de tiempo específico.

In [0]:
import time

#Inicializamos el producer 
producer = KafkaProducer(bootstrap_servers='localhost:9092')

#Contadores que marcan el momento en el cual comenzar a enviar datos
contador = 0
k = 15        

#Arreglos que fungirán como almacén del flujo de datos para cada divisa
flujo_btc = []
flujo_usd = []
flujo_eur = []

#Peridos de las ventanas deslizantes
senal_larga = 30
senal_corta = 10

#Contador del número de datos recibidos
i = 0

#Simulación del flujo de datos
while True:
    #Request de los datos
    response = requests.get(api_url)
    json_ = response.json() #Se almacena en un json
    print(str(json_["ETH"]).encode('utf-8')) #Imprimimos la información
        
    # Anexamos a los arreglos el valor del Ethereum en la divida mostrada
    flujo_btc.append(json_['ETH']['BTC']) #Bitcoin
    flujo_usd.append(json_['ETH']['USD']) #Dólares
    flujo_eur.append(json_['ETH']['EUR']) #Euros
  
    # Tan pronto como exista el número de datos suficientes para calcular la señal larga
    if contador >= senal_larga:         
        #Cálculo de la media móvil (sumando del valor i al i+señal larga)
        promedio_btc = sum(flujo_btc[i:senal_larga+i])/ senal_larga         
        promedio_usd = sum(flujo_usd[i:senal_larga+i])/ senal_larga   
        promedio_eur = sum(flujo_eur[i:senal_larga+i])/ senal_larga   
        
        #Cálculo de la señal corta
        promedio_btc_corta = sum(flujo_btc[i+senal_larga-senal_corta:senal_larga+i])/ senal_corta         
        promedio_usd_corta = sum(flujo_usd[i+senal_larga-senal_corta:senal_larga+i])/ senal_corta   
        promedio_eur_corta = sum(flujo_eur[i+senal_larga-senal_corta:senal_larga+i])/ senal_corta   
            
        #Mostramos los datos
        print(f"Señal larga: {promedio_btc}, {promedio_usd}, {promedio_eur}")
        print(f"Señal corta: {promedio_btc_corta}, {promedio_usd_corta}, {promedio_eur_corta}")
        
        # Anexamos, respectivamente, el valor de la señal larga, los valores actuales y la señal corta
        promedios_dict = dict(prom_btc=promedio_btc,
                              prom_usd=promedio_usd,
                              prom_eur=promedio_eur,
                              btc=json_['ETH']['BTC'],
                              usd=json_['ETH']['USD'],
                              eur=json_['ETH']['EUR'],
                              prom_btc_corta=promedio_btc_corta,
                              prom_usd_corta=promedio_usd_corta,
                              prom_eur_corta=promedio_eur_corta                              
                             ) 
        
        #Enviamos el diccionario para su posterior análisis
        producer.send("prom", str(promedios_dict).encode('utf-8'))                              
        #Aumentamos los contadores
        i += 1        
    contador += 1
    #time.sleep(5)

b"{'BTC': 0.06662, 'USD': 2023.33, 'EUR': 1909.62}"
b"{'BTC': 0.06662, 'USD': 2023.06, 'EUR': 1909.78}"
b"{'BTC': 0.06662, 'USD': 2022.85, 'EUR': 1909.62}"
b"{'BTC': 0.06663, 'USD': 2024.52, 'EUR': 1909.78}"
b"{'BTC': 0.06663, 'USD': 2024.57, 'EUR': 1909.78}"
b"{'BTC': 0.06662, 'USD': 2023.47, 'EUR': 1910.92}"
b"{'BTC': 0.06663, 'USD': 2023.55, 'EUR': 1909.78}"
b"{'BTC': 0.06662, 'USD': 2024.8, 'EUR': 1910.89}"
b"{'BTC': 0.06662, 'USD': 2025.26, 'EUR': 1911.56}"
b"{'BTC': 0.06662, 'USD': 2023.5, 'EUR': 1911.56}"
b"{'BTC': 0.06662, 'USD': 2023.52, 'EUR': 1909.78}"
b"{'BTC': 0.06662, 'USD': 2025.26, 'EUR': 1911.79}"
b"{'BTC': 0.06662, 'USD': 2025.06, 'EUR': 1911.79}"
b"{'BTC': 0.06662, 'USD': 2025.26, 'EUR': 1911.56}"
b"{'BTC': 0.06662, 'USD': 2025.08, 'EUR': 1911.56}"
b"{'BTC': 0.06662, 'USD': 2025.26, 'EUR': 1911.87}"
b"{'BTC': 0.06662, 'USD': 2025.08, 'EUR': 1911.79}"
b"{'BTC': 0.06662, 'USD': 2025.5, 'EUR': 1911.59}"
b"{'BTC': 0.06662, 'USD': 2025.5, 'EUR': 1911.59}"
b"{'BTC': 0.0666

In [0]:
#Muestreo presa.

import numpy as np

#Definimos la clase Muestreo Presa como se vio en clase
class MuestreoPresa:
    def __init__(self, k):
        self.k = k
        self.i = 0
        self.presa = np.zeros(self.k)

    def __call__(self, x):
        if self.i < self.k:
            self.presa[self.i] = x
        else:
            j = np.random.randint(0, self.i)            
            if j < self.k:
                self.presa[j] = x
        self.i += 1

    def obtener_presa(self):
        return self.presa

In [0]:
import time

producer = KafkaProducer(bootstrap_servers='localhost:9092')

contador = 0
k = 15        

#Definimos un muestreo por cada tipo de cambio
mp_btc = MuestreoPresa(k)
mp_usd = MuestreoPresa(k)
mp_eur = MuestreoPresa(k)

#Simulamos el flujo
while True:
    response = requests.get(api_url)
    #Convertimos los datos a un json
    json_ = response.json()
    print(str(json_["ETH"]).encode('utf-8')) #Mostramos los datos
    
    # Aplicamos el algoritmo a los datos obtenidos
    mp_btc(json_['ETH']['BTC'])
    mp_usd(json_['ETH']['USD'])
    mp_eur(json_['ETH']['EUR'])
    
    # Algoritmo VENTANAS DE SALTOS DE TAMAÑO CONSTANTE 
    if contador > 30: #Contador sobre el número de datos procesados

        muestra_btc = mp_btc.obtener_presa()
        muestra_usd = mp_usd.obtener_presa()
        muestra_eur = mp_eur.obtener_presa()

        # Obtencion de los promedios
        promedio_btc = np.mean(muestra_btc)
        promedio_usd = np.mean(muestra_usd)
        promedio_eur = np.mean(muestra_eur)
        
        #Imprimimos los promedios
        print(f"{promedio_btc}, {promedio_usd}, {promedio_eur}")
        promedios_dict = dict(prom_btc=promedio_btc,
                              prom_usd=promedio_usd,
                              prom_eur=promedio_eur
                             )      
        #Guardamos en un diccionario y se envían  
        producer.send("prom", str(promedios_dict).encode('utf-8'))         
        
        #Volvemos al estado inicial 
        mp_btc = MuestreoPresa(k)
        mp_usd = MuestreoPresa(k)
        mp_eur = MuestreoPresa(k)
        contador = 0        
    contador += 1
    #time.sleep(5)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e588c84d-c012-4c07-8593-b08d1bf1b087' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>